In [1]:
import duckdb, pandas as pd, numpy as np, matplotlib.pyplot as plt
from exp_engine.engine.duck import create_connection_and_views

BASE_OUT = "experiments/deletion_capacity/results_parquet"   # <- your exp_engine base_out

conn = create_connection_and_views(BASE_OUT)
conn.execute("PRAGMA threads=8;")
conn.execute("PRAGMA enable_progress_bar=false;")

# Enrich the seeds view with derived metrics (regret_ratio, pass flag, rho_util)
conn.execute("""
CREATE OR REPLACE VIEW seeds_enriched AS
SELECT
  s.*,
  -- choose your regret field name; keeping 'avg_regret_empirical' per exp_engine examples
  avg_regret_empirical AS avg_regret,
  CASE WHEN gamma_bar IS NOT NULL AND gamma_bar != 0
       THEN avg_regret_empirical / gamma_bar END AS regret_ratio,
  CASE WHEN avg_regret_empirical IS NOT NULL AND gamma_bar IS NOT NULL
       THEN (avg_regret_empirical <= gamma_bar) END AS pass_overall,
  CASE WHEN rho_total IS NOT NULL AND rho_total > 0 AND rho_spent_final IS NOT NULL
       THEN rho_spent_final / rho_total END AS rho_util
FROM seeds s;
""")

# Optional: events-level convenience view (keeps only light columns for quick ops)
conn.execute("""
CREATE OR REPLACE VIEW events_light AS
SELECT grid_id, seed, op, regret, gamma_bar, gamma_split
FROM events;
""")


CatalogException: Catalog Error: Table with name events does not exist!
Did you mean "seeds"?

LINE 4: FROM events;
             ^

In [ ]:
# Create a ready-to-run Jupyter notebook template for DuckDB-based analysis
import json, os, textwrap, datetime, uuid, sys

nb = {
  "nbformat": 4,
  "nbformat_minor": 5,
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": sys.version.split()[0]
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# DuckDB Analysis Template for `exp_engine`\n",
        "\n",
        "This notebook connects directly to your **HIVE-partitioned Parquet datasets** written by the experiment engine, builds **instant DuckDB views**, and runs **fast SQL-first analysis** with light Python plotting.\n",
        "\n",
        "**Highlights**\n",
        "- Keep the heavy data in Parquet; pull back only small aggregates\n",
        "- Derived metrics view: `seeds_enriched` (regret ratio, pass flags, ρ-utilization)\n",
        "- Ready-made queries: overall, by-grid, by-controls, ECDF, violin, interactions\n",
        "- Privacy diagnostics: ρ-utilization + per-op γ-split checks\n",
        "- (Optional) Persist small report tables in DuckDB for dashboards\n",
        "\n",
        "> **Note:** Make sure your results path below points at the folder where `exp_engine` wrote `seeds/` and `events/`.\n"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# --- Prerequisites (uncomment if needed) ---\n",
        "# %pip install duckdb pandas matplotlib\n",
        "# %pip install exp_engine  # if distributed as a package; else ensure it's on PYTHONPATH\n",
        "\n",
        "import duckdb, pandas as pd, numpy as np, matplotlib.pyplot as plt\n",
        "from exp_engine.engine.duck import create_connection_and_views\n",
        "\n",
        "plt.rcParams.update({'figure.figsize': (5,3), 'axes.grid': True})\n",
        "\n",
        "BASE_OUT = \"results/parquet\"  # <--- change if your Parquet base_out lives elsewhere\n",
        "\n",
        "conn = create_connection_and_views(BASE_OUT)\n",
        "conn.execute(\"PRAGMA threads=8;\")\n",
        "conn.execute(\"PRAGMA enable_progress_bar=false;\")\n",
        "\n",
        "print(\"Connected. Views available:\")\n",
        "print(conn.execute(\"SHOW TABLES;\").df())"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# --- Derived view with common metrics ---\n",
        "conn.execute(\n",
        "    \"\"\"\n",
        "    CREATE OR REPLACE VIEW seeds_enriched AS\n",
        "    SELECT\n",
        "      s.*,\n",
        "      -- choose your key regret field; exp_engine demo uses 'avg_regret_empirical'\n",
        "      avg_regret_empirical AS avg_regret,\n",
        "      CASE WHEN gamma_bar IS NOT NULL AND gamma_bar != 0\n",
        "           THEN avg_regret_empirical / gamma_bar END AS regret_ratio,\n",
        "      CASE WHEN avg_regret_empirical IS NOT NULL AND gamma_bar IS NOT NULL\n",
        "           THEN (avg_regret_empirical <= gamma_bar) END AS pass_overall,\n",
        "      CASE WHEN rho_total IS NOT NULL AND rho_total > 0 AND rho_spent_final IS NOT NULL\n",
        "           THEN rho_spent_final / rho_total END AS rho_util\n",
        "    FROM seeds s;\n",
        "    \"\"\"\n",
        ")\n",
        "\n",
        "conn.execute(\n",
        "    \"\"\"\n",
        "    CREATE OR REPLACE VIEW events_light AS\n",
        "    SELECT grid_id, seed, op, regret, gamma_bar, gamma_split\n",
        "    FROM events;\n",
        "    \"\"\"\n",
        ")\n",
        "\n",
        "print(conn.execute(\"DESCRIBE seeds_enriched;\").df().head())"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1) Sanity checks"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "print(conn.execute(\"SELECT COUNT(*) AS n_seeds FROM seeds_enriched;\").df())\n",
        "print(conn.execute(\"SELECT COUNT(*) AS n_events FROM events;\").df())\n",
        "conn.execute(\"SELECT * FROM seeds_enriched LIMIT 5;\").df()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2) Core summaries"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Overall summary\n",
        "overall = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      COUNT(*)                                         AS n_seeds,\n",
        "      SUM(CASE WHEN regret_ratio IS NOT NULL THEN 1 ELSE 0 END) AS n_with_ratio,\n",
        "      AVG(CASE WHEN pass_overall THEN 1 ELSE 0 END)    AS pass_rate_overall,\n",
        "      quantile_cont(regret_ratio, 0.5)                 AS median_r,\n",
        "      quantile_cont(regret_ratio, 0.9)                 AS p90_r\n",
        "    FROM seeds_enriched;\n",
        "    \"\"\"\n",
        ").df()\n",
        "overall"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Loosest grids (lowest median r)\n",
        "by_grid = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      grid_id,\n",
        "      COUNT(*)                                          AS n,\n",
        "      AVG(CASE WHEN pass_overall THEN 1 ELSE 0 END)     AS pass_rate,\n",
        "      quantile_cont(regret_ratio, 0.5)                  AS median_r,\n",
        "      quantile_cont(regret_ratio, 0.9)                  AS p90_r\n",
        "    FROM seeds_enriched\n",
        "    GROUP BY 1\n",
        "    ORDER BY median_r ASC, p90_r ASC, n DESC\n",
        "    LIMIT 20;\n",
        "    \"\"\"\n",
        ").df()\n",
        "by_grid.head(10)"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# By-controls summary (customize columns to the knobs you varied)\n",
        "by_ctrl = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      gamma_bar, gamma_split, rho_total, delete_ratio, target_PT,\n",
        "      COUNT(*)                                      AS n,\n",
        "      AVG(CASE WHEN pass_overall THEN 1 ELSE 0 END) AS pass_rate,\n",
        "      quantile_cont(regret_ratio, 0.5)              AS median_r,\n",
        "      quantile_cont(regret_ratio, 0.9)              AS p90_r\n",
        "    FROM seeds_enriched\n",
        "    GROUP BY 1,2,3,4,5\n",
        "    ORDER BY median_r ASC, p90_r ASC, n DESC\n",
        "    LIMIT 200;\n",
        "    \"\"\"\n",
        ").df()\n",
        "by_ctrl.head(20)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3) ECDF and violin plots\n",
        "Pull only small, aggregated data for plotting—let DuckDB do the heavy lifting."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# ECDF of regret_ratio\n",
        "ecdf = conn.execute(\n",
        "    \"\"\"\n",
        "    WITH vals AS (\n",
        "      SELECT regret_ratio AS r FROM seeds_enriched WHERE regret_ratio IS NOT NULL\n",
        "    ),\n",
        "    ord AS (\n",
        "      SELECT r,\n",
        "             ROW_NUMBER() OVER (ORDER BY r) AS rn,\n",
        "             COUNT(*)  OVER ()              AS n\n",
        "      FROM vals\n",
        "    )\n",
        "    SELECT r, CAST(rn AS DOUBLE)/n AS ecdf\n",
        "    FROM ord\n",
        "    ORDER BY r;\n",
        "    \"\"\"\n",
        ").df()\n",
        "\n",
        "plt.figure()\n",
        "plt.plot(ecdf[\"r\"], ecdf[\"ecdf\"])\n",
        "plt.axvline(1.0, linestyle='--', linewidth=1)\n",
        "plt.title(\"Regret Ratio ECDF\")\n",
        "plt.xlabel(\"regret_ratio\")\n",
        "plt.ylabel(\"F(r)\")\n",
        "plt.grid(alpha=0.2)\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Violin by a factor (example: delete_ratio)\n",
        "samples = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT CAST(delete_ratio AS DOUBLE) AS factor, regret_ratio\n",
        "    FROM seeds_enriched\n",
        "    WHERE regret_ratio IS NOT NULL AND delete_ratio IS NOT NULL;\n",
        "    \"\"\"\n",
        ").df()\n",
        "\n",
        "cats = sorted(samples[\"factor\"].dropna().unique())\n",
        "data = [samples.loc[samples[\"factor\"]==c, \"regret_ratio\"].values for c in cats]\n",
        "\n",
        "plt.figure()\n",
        "plt.violinplot(data, showmeans=True, showmedians=True)\n",
        "plt.xticks(range(1, len(cats)+1), [str(c) for c in cats])\n",
        "plt.axhline(1.0, linestyle='--', linewidth=1)\n",
        "plt.title(\"Regret Ratio by delete_ratio\")\n",
        "plt.xlabel(\"delete_ratio\")\n",
        "plt.ylabel(\"regret_ratio\")\n",
        "plt.grid(alpha=0.2)\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4) Effects & interactions"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Global baselines for deltas\n",
        "global_stats = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      quantile_cont(regret_ratio, 0.5) AS median_r,\n",
        "      quantile_cont(regret_ratio, 0.9) AS p90_r\n",
        "    FROM seeds_enriched\n",
        "    WHERE regret_ratio IS NOT NULL;\n",
        "    \"\"\"\n",
        ").df().iloc[0]\n",
        "g_med, g_p90 = float(global_stats.median_r), float(global_stats.p90_r)\n",
        "g_med, g_p90"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Marginal effects by control\n",
        "marginals = conn.execute(\n",
        "    \"\"\"\n",
        "    WITH marg AS (\n",
        "      SELECT 'gamma_bar'     AS control, CAST(gamma_bar     AS DOUBLE) AS level,\n",
        "             COUNT(*) AS n, quantile_cont(regret_ratio,0.5) AS med, quantile_cont(regret_ratio,0.9) AS p90\n",
        "        FROM seeds_enriched WHERE regret_ratio IS NOT NULL GROUP BY 2\n",
        "      UNION ALL\n",
        "      SELECT 'gamma_split',  CAST(gamma_split  AS DOUBLE), COUNT(*), quantile_cont(regret_ratio,0.5), quantile_cont(regret_ratio,0.9)\n",
        "        FROM seeds_enriched WHERE regret_ratio IS NOT NULL GROUP BY 2\n",
        "      UNION ALL\n",
        "      SELECT 'target_PT',    CAST(target_PT    AS DOUBLE), COUNT(*), quantile_cont(regret_ratio,0.5), quantile_cont(regret_ratio,0.9)\n",
        "        FROM seeds_enriched WHERE regret_ratio IS NOT NULL GROUP BY 2\n",
        "      UNION ALL\n",
        "      SELECT 'delete_ratio', CAST(delete_ratio AS DOUBLE), COUNT(*), quantile_cont(regret_ratio,0.5), quantile_cont(regret_ratio,0.9)\n",
        "        FROM seeds_enriched WHERE regret_ratio IS NOT NULL GROUP BY 2\n",
        "      UNION ALL\n",
        "      SELECT 'rho_total',    CAST(rho_total    AS DOUBLE), COUNT(*), quantile_cont(regret_ratio,0.5), quantile_cont(regret_ratio,0.9)\n",
        "        FROM seeds_enriched WHERE regret_ratio IS NOT NULL GROUP BY 2\n",
        "    )\n",
        "    SELECT control, level, n, med, p90\n",
        "    FROM marg\n",
        "    ORDER BY control, med ASC, p90 ASC;\n",
        "    \"\"\"\n",
        ").df()\n",
        "\n",
        "marginals[\"abs_med_delta\"] = (marginals[\"med\"] - g_med).abs()\n",
        "marginals.sort_values([\"abs_med_delta\",\"control\"], ascending=[False, True]).head(12)"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Interaction heatmap: gamma_bar × target_PT\n",
        "heat = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      CAST(gamma_bar AS DOUBLE) AS gamma_bar,\n",
        "      CAST(target_PT AS DOUBLE) AS target_PT,\n",
        "      quantile_cont(regret_ratio, 0.5) AS median_r\n",
        "    FROM seeds_enriched\n",
        "    WHERE regret_ratio IS NOT NULL\n",
        "    GROUP BY 1,2\n",
        "    ORDER BY 1,2;\n",
        "    \"\"\"\n",
        ").df()\n",
        "\n",
        "if not heat.empty:\n",
        "    pivot = heat.pivot(index=\"gamma_bar\", columns=\"target_PT\", values=\"median_r\")\n",
        "    plt.figure(figsize=(5,3))\n",
        "    im = plt.imshow(pivot.values, aspect='auto')\n",
        "    plt.xticks(range(len(pivot.columns)), [str(c) for c in pivot.columns], rotation=45, ha='right')\n",
        "    plt.yticks(range(len(pivot.index)),  [str(i) for i in pivot.index])\n",
        "    plt.colorbar(im, fraction=0.046, pad=0.04)\n",
        "    plt.title(\"Median regret ratio: gamma_bar × target_PT\")\n",
        "    for yi in range(pivot.shape[0]):\n",
        "        for xi in range(pivot.shape[1]):\n",
        "            val = pivot.values[yi, xi]\n",
        "            if np.isfinite(val):\n",
        "                plt.text(xi, yi, f\"{val:.2f}\", ha='center', va='center', fontsize=8, color='white')\n",
        "    plt.tight_layout(); plt.show()\n",
        "else:\n",
        "    print(\"No data for interaction heatmap.\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5) Privacy diagnostics"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# ρ utilization summary\n",
        "rho_stats = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      COUNT(*)                                        AS n_with_rho,\n",
        "      quantile_cont(rho_util, 0.5)                    AS rho_util_median,\n",
        "      quantile_cont(rho_util, 0.9)                    AS rho_util_p90,\n",
        "      MAX(rho_util)                                   AS rho_util_max\n",
        "    FROM seeds_enriched\n",
        "    WHERE rho_util IS NOT NULL;\n",
        "    \"\"\"\n",
        ").df()\n",
        "rho_stats"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Scatter: rho_util vs regret_ratio\n",
        "rho_scatter = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT rho_util, regret_ratio\n",
        "    FROM seeds_enriched\n",
        "    WHERE rho_util IS NOT NULL AND regret_ratio IS NOT NULL;\n",
        "    \"\"\"\n",
        ").df()\n",
        "\n",
        "if not rho_scatter.empty:\n",
        "    plt.figure()\n",
        "    plt.scatter(rho_scatter[\"rho_util\"], rho_scatter[\"regret_ratio\"], s=12, alpha=0.5)\n",
        "    plt.xlabel(\"rho_util = rho_spent_final / rho_total\")\n",
        "    plt.ylabel(\"regret_ratio\")\n",
        "    plt.title(\"Privacy spend vs regret ratio\")\n",
        "    plt.grid(alpha=0.2)\n",
        "    plt.show()\n",
        "else:\n",
        "    print(\"No rho/util or regret_ratio data to plot.\")"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "# Per-op mean regret and γ-split pass checks (events-level)\n",
        "perop = conn.execute(\n",
        "    \"\"\"\n",
        "    SELECT\n",
        "      grid_id,\n",
        "      AVG(CASE WHEN op='insert' THEN regret END) AS insert_mean,\n",
        "      AVG(CASE WHEN op='delete' THEN regret END) AS delete_mean,\n",
        "      ANY_VALUE(gamma_bar)   AS gamma_bar,\n",
        "      ANY_VALUE(gamma_split) AS gamma_split\n",
        "    FROM events_light\n",
        "    GROUP BY 1;\n",
        "    \"\"\"\n",
        ").df()\n",
        "\n",
        "if not perop.empty:\n",
        "    perop[\"insert_pass\"] = perop[\"insert_mean\"] <= perop[\"gamma_bar\"] * perop[\"gamma_split\"]\n",
        "    perop[\"delete_pass\"] = perop[\"delete_mean\"] <= perop[\"gamma_bar\"] * (1 - perop[\"gamma_split\"])\n",
        "    perop.head(10)\n",
        "else:\n",
        "    print(\"No events data available for per-op checks.\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6) Persist small report tables (optional)\n",
        "Use these as sources for dashboards or to avoid recomputing summaries."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "conn.execute(\"CREATE SCHEMA IF NOT EXISTS reports;\")\n",
        "\n",
        "conn.execute(\n",
        "    \"\"\"\n",
        "    CREATE OR REPLACE TABLE reports.overall AS\n",
        "    SELECT * FROM (\n",
        "      SELECT\n",
        "        COUNT(*)                                         AS n_seeds,\n",
        "        AVG(CASE WHEN pass_overall THEN 1 ELSE 0 END)    AS pass_rate_overall,\n",
        "        quantile_cont(regret_ratio, 0.5)                 AS median_r,\n",
        "        quantile_cont(regret_ratio, 0.9)                 AS p90_r\n",
        "      FROM seeds_enriched\n",
        "    );\n",
        "    \"\"\"\n",
        ")\n",
        "\n",
        "conn.execute(\n",
        "    \"\"\"\n",
        "    CREATE OR REPLACE TABLE reports.by_controls AS\n",
        "    SELECT\n",
        "      gamma_bar, gamma_split, rho_total, delete_ratio, target_PT,\n",
        "      COUNT(*)                                      AS n,\n",
        "      AVG(CASE WHEN pass_overall THEN 1 ELSE 0 END) AS pass_rate,\n",
        "      quantile_cont(regret_ratio, 0.5)              AS median_r,\n",
        "      quantile_cont(regret_ratio, 0.9)              AS p90_r\n",
        "    FROM seeds_enriched\n",
        "    GROUP BY 1,2,3,4,5;\n",
        "    \"\"\"\n",
        ")\n",
        "\n",
        "print(conn.execute(\"SELECT * FROM reports.overall;\").df())\n",
        "print(conn.execute(\"SELECT * FROM reports.by_controls LIMIT 10;\").df())"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7) Export light CSVs (optional)\n",
        "Write out small CSVs for sharing or quick diffs."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": None,
      "outputs": [],
      "source": [
        "EXPORT_DIR = \"exports\"  # change if desired\n",
        "os.makedirs(EXPORT_DIR, exist_ok=True)\n",
        "\n",
        "overall.to_csv(os.path.join(EXPORT_DIR, \"overall.csv\"), index=False)\n",
        "by_grid.to_csv(os.path.join(EXPORT_DIR, \"by_grid_top20.csv\"), index=False)\n",
        "by_ctrl.to_csv(os.path.join(EXPORT_DIR, \"by_controls.csv\"), index=False)\n",
        "print(\"Wrote:\", os.listdir(EXPORT_DIR))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "### Next steps\n",
        "- Add your custom KPIs to `seeds_enriched` once and reuse across queries.\n",
        "- If events are huge, sample with `WHERE seed IN (...)` or add time windows.\n",
        "- Create more `reports.*` tables for dashboards or save plots to disk for CI artifacts."
      ]
    }
  ]
}

out_path = "/mnt/data/duckdb_exp_engine_analysis_template.ipynb"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(nb, f, indent=2)

out_path


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/duckdb_exp_engine_analysis_template.ipynb'